Please fill in your name and that of your teammate.

You: Albin Aliu

Teammate: Christoph Jutzet

# Introduction

Welcome to the fifth lab. Last week we had a break from math-heavy assignments to allow you to catch up on the first 3 assignments. As far as learning the basics of Pandas could be considered a break anyway. We will learn more of this library over the weeks, hope you will develop an appreciation as you build up experience. "You may find its methods disagreeable, but you can't avoid appreciating the results" (cit.)

## Grouping

You should have some fundamental experience on all libraries used so far at this point (keep the past exercises at hand as reference). It is time to introduce to you the math applications of Pandas DataFrame and Series, and to the unfriendly but oh-so-useful `groupby()`.

From now on we will be using Pandas for our main data structures, even in the math calculation. Remember that they wrap around Numpy arrays while giving convenient indexing and extra capabilities. No need to e.g. split the points depending on the class into a `dict` as we did before: we can _group_ data by label, then all operations will work on the whole feature arrays and for all classes at once (and running the underlying, optimized C implementation).
Simply treat a DataFrame just like you would a multi-dimensional Numpy array (and a Series as if it was a one-dimensional Numpy array). Function calls will be _broadcasted_ to its elements.

We will use `groupby()` extensively. It can be slow to grasp at first, but your code will be legible and you will need (almost) no more loops nor `dict`s. 
Careful because it returns [a `GroupBy` object](https://pandas.pydata.org/pandas-docs/stable/reference/groupby.html) that is somehow unwieldy: it removes a feature, adds a dimension (the grouping), and does not print directly. Try to follow it with a `describe()` to see its application, or print the `groups` for an intuition on how it works: basically a `dict` from each element of the group (e.g. the classes) to the indices of the corresponding rows. Basically it's just an implicit split on the data, and now any operation you call on the GroupBy object will return multiple results (one per group) instead of just one. Go ahead and give it a try, understanding this is necessary before moving on to the next questions, and it's easier if you play with it yourself: the trick that did it for me was to try and ignore its output *per se*, and instead just call functions on it such as `describe()`. 

You need to start thinking of the data as a whole, high-dimensional entity, and your exploration as selecting and slicing this object from different perspectives as if you were "floating around it in space". When using the DataFrame for math just remember that you are manipulating multiple variables at the same time: treat it like a special Numpy data structure and everything should be intuitive.

### How to pass the lab?

Below you find the exercise questions. Each question awarding points is numbered and states the number of points like this: **[0pt]**. To answer a question, fill the cell below with your answer (markdown for text, code for implementation). Incorrect or incomplete answers are in principle worth 0 points: to assign partial reward is only up to teacher discretion. Over-complete answers do not award extra points (though they are appreciated and will be kept under consideration). Save your work frequently! (`ctrl+s`)

**You need at least 12 points (out of 18 available) to pass** (66%).

In [0]:
# Let me hit ctrl+c ctrl+v for you
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
sns.set(rc={'figure.figsize':(8,6)}, style="whitegrid")

# 1. Fundamentals

This time we start strong with an example that is simple but longer. Take your time to read and understand each part, follow the suggestions, and it should unravel without much trouble.

#### 1.1 **[4pt]** When the weather forecast says it is going to rain, 15% of the time they are wrong. When they say it is not going to rain, they are wrong 5% of the times. Also, hypothesize that in the current season you get rain on 20% of the days. Using Bayes' rule, calculate by hand the probability of rain for a day, given that the forecast said it is going to rain.

I suggest you proceed as follows: (i) fill the data you know in an events probability [table](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables), as seen in the lecture; (ii) remember that probabilities sum to a constant over all possible events, so (make a copy of it and) fill in the blanks; (iii) state very clearly what are the posterior, prior, likelihood and evidence; (iv) only assemble your Bayes' equation once you are certain of your components.

Let's translate first our human readable data into math equations:

- Let $A =$ "Forecast says it's going to rain" (hypothesis)
- Let $B =$ "It's going to rain" (observation)

Now, 
"When the weather forecast says it is going to rain, 15% of the time they are wrong." means:
- $P(not B|A) = 0.15$ 

And, similarly: "When they say it is not going to rain, they are wrong 5% of the times." translates into
- $P(B| not A) = 0.05$

And, also "Also, hypothesize that in the current season you get rain on 20% of the days."
- $P(B) = 0.2$


Which gives us:

| -                     | B (rain)      | not B       | total  |
|:---------------------:|:-------------:|:-----------:|:------:|
| **A (forecast rain)** | x             | 0.15        | x+0.15 |
| **not A**             | 0.05          | y           | y+0.05 |
| **total**				| 0.2	  		| 0.8    	  | 1      |


Filling the gaps such that the rows and columns sum up the right way, gives us:

| -                     | B (rain)      | not B       | total  |
|:---------------------:|:-------------:|:-----------:|:------:|
| **A (forecast rain)** | 0.15          | 0.15        | 0.3    |
| **not A**                | 0.05          | 0.65        | 0.7    |
| **total**			| 0.2	  		| 0.8     	  | 1      |


And finally, 

$$
P(B|A) = \frac{P(B,A)}{P(B)} = \frac{P(B)\cdot P(A|B)}{P(A)} = \frac{ 0.2\cdot \frac{0.15}{0.2}}{0.3} = 0.50
$$

#### 1.2 **[1pt]** Explain $\hat{y} = \text{arg}\!\max_{y \in Y}\big\{P(y \,|\, x)\big\}$ .

As usual, $\hat y$ is the prediction, the predicted label. When we say $argmax_{x\in Y}$, we take one particular $y \in Y$, for which we have that the function value of $P(y|x)$ is maximal. Thus take that $y \in Y$ for which the probability to happen given $x$ happened is the highest of all $y \in Y$. 

This way, we take that label with the highest probability (to happen) and minimize the mistake we can make (statistically).

#### 1.3 **[1pt]** How does NB differ from LDA in maintaining the covariance of the distributions modeling the data?


Using NB we lose the "dependence" of the features, information relating two features together. LDA maintains the covariance between the features. But as seen in the lecture, citation: *We lose a bit of the modeling power in exchange for a lot more flexibility and performance.*

# 2. Model Selection for Naïve Bayes

#### 2.1 **[3pt]** Load the `tips` dataset from Seaborn  (into a Pandas DataFrame). Which distribution would you use to model each of the features in the dataset? Explain your choices.

You load the dataset the same way you did for `iris` before. Obviously you need to study it to be able to answer. You should find useful to consider (i) the list of dtypes for each feature, (ii) the number of unique values for each of the categorical features, (iii) you can use the `pairplot` to quickly inspect the data: can you do better than a simple Gaussian if there are multiple peaks or asymmetry in the distribution of the real-valued features?  
The code cell below is to hold your analysis, while the real answer + motivations go in the Markdown cell just underneath.

In [10]:
pd.set_option('display.max_rows', None)
tips = sns.load_dataset("tips")
sns.set(rc={'figure.figsize':(8,6)}, style="whitegrid")



#dtypes to consider
print(tips.dtypes)
#unique values
print(tips.nunique(axis = 0))

## To identify distributions, use this!
#cat_columns = tips.select_dtypes(['category']).columns
#tips[cat_columns] = tips[cat_columns].apply(lambda x: x.cat.codes)
#sns.pairplot(tips)
#
## To look at the real-valued fearures, distinct between them with hue
#sns.pairplot(tips, hue="time")
#sns.pairplot(tips, hue="sex")
#sns.pairplot(tips, hue="smoker")
#sns.pairplot(tips, hue="day")



total_bill     float64
tip            float64
sex           category
smoker        category
day           category
time          category
size             int64
dtype: object
total_bill    229
tip           123
sex             2
smoker          2
day             4
time            2
size            6
dtype: int64


Use the first part to visually see the distributions for the discrete features:

- total_bill, tip (real valued): simple gaussian/normal
- sex, smoker, time (binary): bernoulli
- day, size (discreet, more than two): binomial

And then use the second part with the hue to see that we have mixture-gaussians, where we could model the distribution with two or more gaussian distributions for the real-valued features total_bill and tip.

# 3. Naïve Bayes Classification

Let's write a Naïve Bayes classifier from scratch. We will work with the `iris` dataset (again, from Seaborn) since we know already the data. All features are continuous: for simplicity we can use simple Gaussians, but we should expect some misclassification.

From now on let's also introduce the train-test split so we can start verifying our model's performance the right way. Just use `train` for your answer instead of `df`, and leave `test` for the end.

In [0]:
df = sns.load_dataset('iris')

from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.2) # 80-20 split

#### 3.1 **[2pt]** Compute the priors for the three classes of the Iris dataset using the Pandas DataFrame, _in a single line of code_ and without using loops (`for`, `while`, etc.).

One-liners are typically bad practice, but here I need to force you to learn this new tool and stop writing `for` loops, since they will not scale from now on.  

Careful as many tutorials online (such as [this one](https://chrisalbon.com/machine_learning/naive_bayes/naive_bayes_classifier_from_scratch/) will explicitly select the class and run the same calculation multiple times (and in multiple lines). This approach **does not scale** to problems with 100 or 10'000 classes: learn to use `groupby()` instead!  
_[Think: this course should make you confident enough to be the one writing the tutorials, and hopefully of much better quality!]_

As a reference, you will need to (i) group the dataframe by species, (ii) select only the grouped elements (returning a Series), (iii) run the Numpy-backed `count()`, (iv) divide by the total number of elements.  

In [0]:
priors = train.groupby(['species']).size().div(train.shape[0])



#### 3.2 **[1pt]** Compute the means and the standard deviations for each feature and for each class of the Iris dataset using the Pandas DataFrame (one line of code each).

As a reference, you should obtain 12 means and 12 standard deviations. Again, the use of `groupby` followed by Numpy's functions will take literally 2 lines and no loops. Remember to use the `train` data!

In [0]:
stds = train.groupby(['species']).std();
means = train.groupby(['species']).mean();

Here is a freebie to save you some debugging time: the (stunted) equation for the Gaussian probability. Stunted in the sense that, since it is only used to maximize the class probability, parts that do not depend on the class have been dropped (as usual). It requires you to define first the variables `means` and `stds` from the previous question (both $(3\times4)$ DataFrames).

If you really want to understand what is going on (especially with Pandas), I challenge you to comment it out, pull the slides, and write your own. You did something very close for LDA, feel free to review your code. You don't need it to look identical as long as it does the same job.

Remember that Naïve Bayes computes the class likelihood as a product of the independent probabilities for each feature: this is done by the `product()` on the columns. If you remove that, you should have 12 values (give it a try).

When passing a line of input to `likelihood` be careful to remove the last column (the `species`) as in the example below (in our previous calculations this was done by the `groupby()`, which made a new dimension out of it).

In [14]:
likelihood = lambda x: (np.exp(-(x-means)**2/(2*stds**2))/stds).product(axis=1)
likelihood(train.iloc[50, :-1]).round(3)

species
setosa         0.000
versicolor     0.001
virginica     20.650
dtype: float64

#### 3.3 **[1pt]** Write a Python function that predicts the class of input $x$ (i.e. returns $\hat{y}$ for one line of data).

As a sanity check: it should take a row as input (without labels, as for `likelihood` above) and return the string found in the `index` of the max value (the documentation is your friend).

In [16]:
predictor = lambda x: (priors*likelihood(x)).idxmax(axis = 0) 

predictor(train.iloc[50, :-1])

'virginica'

#### 3.4 **[2pt]** Compute $\hat{y}$ for all points in the `test` dataset, in one line and without using Python loops (`for`, `while`, etc.). Compare it with the correct label $y$ and print the number of misclassified points.

And here is how you use the test set: after the training on the train set is complete, you evaluate its performance on data it was not trained on. This is absolutely **crucial** in machine learning. We will use this process from now on, and using the wrong dataset (either for training or testing) will be considered a major error (so careful with typos! Double-check every time!). If you wonder why so strict, check again the 4th lecture and ask yourself what are the consequences of getting it wrong in a work or research setting (and feel free to discuss on Moodle).

Again, no loops: you need both to drop the last column and then to apply the function to the rows. For example: `train.iloc[:, :-1].apply(my_predict_fn, axis = 1)`. Can you make it look nicer/more readable?

Remember you can count the number of `True` values in a numpy array simply by calling `sum()` on it.

In [18]:
predicted = test.iloc[:, :-1].copy().apply(predictor, axis = 1)

matches = np.where(test['species'] == predicted, True, False)

"misclassified",len(test)-sum(matches)

('misclassified', 0)

#### 3.5 **[1pt]** Why did we not compute (nor need) the _evidence_ for predicting the input's class?

Because it does not depend on $y$.
We try to maximize Naïve Bayes, thus we try to maximize:
$$
P(y|x) = \frac{P(y)\cdot P(x|y)}{P(x)}
$$

The evidence $P(x)$ does not depend on $y$, therefore we can drop it. 

Basically, if you have a list:
```
{
  1: 5/9
  2: 4/9
  3: 8/9
}
```


The max of this list is the value with index 3. The same would be true if we would drop the /9 for every entry.

#### 3.6 **[2pt]** Train a scikit-learn Naïve Bayes Gaussian classifier on the Iris train data using a Pandas Dataframe, and print the number of misclassified points on the test data.

Remember that:
- Now that we have a bit more experience with Pandas we can learn how to pass the DataFrames directly to scikit-learn.
- The training data should always be 2D (i.e. DataFrame) and not have the label (`train.iloc[:,:-1]`, do you know what each `:` stands for?).
- The labels should always be 1D (i.e. Series) and numerical. Rather than doing the conversion manually, you should convert the feature to categorical and then use its codes (`train['species'].astype('category').cat.codes`).
- Mistakenly testing on the train set will fail the question, as will comparing the prediction against the train set labels (hint hint).
- You will probably get better results with scikit-learn because it uses multivariate Gaussians and improved estimators (check the [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html)).

In [0]:
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
clf.fit(train.iloc[:,:-1], train['species'])
GaussianNB()

predicted = clf.predict(test.iloc[:,:-1])

np.where(test['species'] == predicted, True, False)

"misclassified",len(test)-sum(matches)

('misclassified', 0)

# At the end of the exercise

Bonus question with no points! Answering this will have no influence on your scoring, not at the assignment and not towards the exam score -- really feel free to ignore it with no consequence. But solving it will reward you with skills that will make the next lectures easier, give you real applications, and will be good practice towards the exam.

The solution for this questions will not be included in the regular lab solutions pdf, but you are welcome to open a discussion on the Moodle: we will support your addressing it, and you may meet other students that choose to solve this, and find a teammate for the next assignment that is willing to do things for fun and not only for score :)

#### BONUS **[ZERO pt]** Do a bit of independent research, and propose below the simplest example you can, to make evident how the frequentist and Bayesian approaches are different.

I advise against blind copy+paste from the Internet in this case, I have seen so many incorrect opinions and tutorials over the years it is frankly ridiculous. I suggest you rather argue a bit on the Moodle about the approaches themselves, so you can make sure your example is correct.

A good intro: [[link]](https://ocw.mit.edu/courses/mathematics/18-05-introduction-to-probability-and-statistics-spring-2014/readings/MIT18_05S14_Reading20.pdf).

#### BONUS **[ZERO pt]** Train a Gaussian NB (either your code or scikit-learn) on the full Iris dataset (no train-test split) and check the misclassifications. Train the same on the 80% training data, then check and aggregate misclassifications both on the train and test datasets. You will probably get the same number of total errors regardless of whether you trained on 80% or 100% of the data. Can you explain why? Feel free to play with different splits until you find how low can you go with the training before increasing the number of errors. Use the term `statistically representative` in your explanation.

### Final considerations

- This is the first core ML method we are covering in the course. As you see it expects you to know quite a lot of concepts before we can really discuss its workings.
- This is also the first method capable of nonlinear classification. As it can work with multiple classes and different types of distributions (think Mixture of Gaussians), the division boundary is not a line anymore.
- In the next two lectures we will ease into the other big gun of core ML, the Support Vector Machine with its infamous Kernel Trick. We are reaching the real complexity level of the course, and we will keep this difficulty until the exam. Follow closely both the lectures and exercises and you should have no trouble.